In [31]:
import torch
import numpy as np

In [5]:
prediction = torch.load('../hpc-tensors/Best_prediction_1.pt')
targets = torch.load('../hpc-tensors/Targes_1.pt')


In [23]:
prediction.size()

torch.Size([5248, 10])

In [22]:
prediction[:,0:1].sum()

tensor(1864)

In [30]:
for i in range(prediction.size()[1]):
    print(f'Pred:\t{prediction[:,i:i+1].sum()}')
    print(f'Target:\t{targets[:,i:i+1].sum()}')
    print()

Pred:	1864
Target:	1880

Pred:	1972
Target:	1718

Pred:	707
Target:	656

Pred:	384
Target:	449

Pred:	686
Target:	745

Pred:	208
Target:	356

Pred:	219
Target:	232

Pred:	38
Target:	239

Pred:	3
Target:	222

Pred:	0
Target:	30



In [35]:
confusionmatrix = np.zeros((10,10))
print(confusionmatrix)
print(prediction.size()[0] == targets.size()[0])

for i in range(prediction.size()[1]):
    for j in range(prediction.size()[0]):
        if prediction[j:j+1,i:i+1] == 1:
            
    print(f'Pred:\t{prediction[:,i:i+1].sum()}')
    print(f'Target:\t{targets[:,i:i+1].sum()}')
    print()


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
True


"\nfor i in range(prediction.size()[1]):\n    for j in range(prediction.size()[0]):\n    print(f'Pred:\t{prediction[:,i:i+1].sum()}')\n    print(f'Target:\t{targets[:,i:i+1].sum()}')\n    print()\n"